In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import os

In [2]:
!pwd

/home/balavarun/Desktop/nlp/edbot/EdBotModel


In [27]:
def ingest_docs(filename):
    path = "docs/" + filename
    if(filename.endswith(".pdf") == False):
        return
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    # print(pages[0].page_content)
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    print(docs[0].metadata)


    # embeddings = OpenAIEmbeddings()
    # db = FAISS.from_documents(documents=docs, embedding=embeddings)

    # #save to pinecone too
    # Pinecone.from_documents(pages, embedding=embeddings, index_name="edbot")


    path_to_index = "docs/faiss_indices/" + filename + ".index"
    # db.save_local(path_to_index)
    return docs

In [28]:
folder_path = "docs/"

for filename in os.listdir(folder_path):
    test = ingest_docs(filename)
    break

# type(test)

{'source': 'docs/qualomm-2.pdf', 'page': 0}
